# Project 2

    In order to better understand how our organization can finance the investments in the Video game industry we previously analyzed, we must first understand the state of our organization.

 

    You have been charged with analyzing the current state of our organization to determine what level of investment we can make in the game industry. To accomplish this you must do the following:

 

    1) How have video game sales performed in our organization as a whole?
        Show the profit for media and gaming for our entire company by month.
    2) What do you feel is the direct cause of the performance of media and gaming?
    3) After evaluating the media and gaming department in our company, what is your recommendation for investment in the video game industry?

In [3]:
from dbConfig import create_conn

conn = create_conn()
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM sales")
result = cursor.fetchone()
print(f"Total sales records: {result[0]}")

cursor.close()
conn.close()

Total sales records: 367213


In [8]:
from dbConfig import create_conn

def get_media_gaming_profit():
    conn = create_conn()
    cursor = conn.cursor()

    query = """
        SELECT 
            r.Month,
            r.TotalRevenue,
            IFNULL(e.TotalExpenses, 0) AS TotalExpenses,
            (r.TotalRevenue - IFNULL(e.TotalExpenses, 0)) AS Profit
        FROM
        (
            SELECT 
                DATE_FORMAT(SalesDate, '%Y-%m') AS Month,
                SUM(Weekly_Sales) AS TotalRevenue
            FROM sales s
            JOIN departments d ON s.Dept = d.Dept
            WHERE d.DeptDescription LIKE '%media%'
               OR d.DeptDescription LIKE '%gaming%'
            GROUP BY Month
        ) r
        LEFT JOIN
        (
            SELECT
                DATE_FORMAT(ExpenseDate, '%Y-%m') AS Month,
                SUM(Amount) AS TotalExpenses
            FROM expenses e
            JOIN expense_type et ON e.GLAccount = et.GLAccount
            WHERE et.Description LIKE '%media%'
               OR et.Description LIKE '%gaming%'
            GROUP BY Month
        ) e ON r.Month = e.Month
        ORDER BY r.Month;
    """

    cursor.execute(query)
    results = cursor.fetchall()

    headers = [desc[0] for desc in cursor.description]


    print("-" * 70)
    print("{:<10} {:>15} {:>15} {:>15}".format(headers[0], headers[1], headers[2], headers[3]))
    print("-" * 70)

    # Print rows
    for row in results:
        print("{:<10} {:>15,.2f} {:>15,.2f} {:>15,.2f}".format(row[0], row[1], row[2], row[3]))

    print("-" * 70)

    cursor.close()
    conn.close()

if __name__ == "__main__":
    get_media_gaming_profit()

----------------------------------------------------------------------
Month         TotalRevenue   TotalExpenses          Profit
----------------------------------------------------------------------
2019-02       3,981,645.25            0.00    3,981,645.25
2019-03       3,312,521.48            0.00    3,312,521.48
2019-04       3,443,900.76            0.00    3,443,900.76
2019-05       2,422,348.82            0.00    2,422,348.82
2019-06       2,597,272.14            0.00    2,597,272.14
2019-07       3,052,202.36            0.00    3,052,202.36
2019-08       2,280,959.77            0.00    2,280,959.77
2019-09       2,276,537.94            0.00    2,276,537.94
2019-10       2,974,144.17            0.00    2,974,144.17
2019-11       5,440,283.51            0.00    5,440,283.51
2019-12       9,601,961.32            0.00    9,601,961.32
2020-01       2,360,557.59            0.00    2,360,557.59
2020-02       3,113,559.88            0.00    3,113,559.88
2020-03       2,856,570.98      

In [9]:
from dbConfig import create_conn

def test_query():
    conn = create_conn()
    cursor = conn.cursor()

    query = """
    SELECT DISTINCT et.GLAccount, et.Description
    FROM expense_type et
    ORDER BY et.Description;
    """

    cursor.execute(query)
    results = cursor.fetchall()

    for row in results:
        print(row)

    cursor.close()
    conn.close()


if __name__ == "__main__":
    test_query()


(925687.0, 'Advertising')
(955000.0, 'Construction')
(952001.0, 'Distribution')
(945625.0, 'Hardware')
(936548.0, 'Insurance')
(953654.0, 'Payroll')
(923655.0, 'Recruiting')
(954832.0, 'Rent')
(936524.0, 'Software')
(952468.0, 'Training')
(952000.0, 'Utilities')


In [14]:
from dbConfig import create_conn
from tabulate import tabulate

def option_1():
    conn = create_conn()
    cursor = conn.cursor()

    query = """
        SELECT 
            SUM(e.Amount) AS TotalAmount,
            et.Description
        FROM expenses e
        JOIN expense_type et 
            ON e.GLAccount = et.GLAccount
        GROUP BY et.Description
        ORDER BY et.Description;
    """

    cursor.execute(query)
    results = cursor.fetchall()

    print("\n=== TOTAL EXPENSES BY TYPE ===")
    print(tabulate(results, headers=["Total Amount", "Description"], floatfmt=",.2f"))

    cursor.close()
    conn.close()


if __name__ == "__main__":
    option_1()


=== TOTAL EXPENSES BY TYPE ===
    Total Amount  Description
----------------  -------------
  372,807,052.83  Advertising
    4,169,330.00  Construction
1,191,843,376.42  Distribution
  101,996,354.50  Hardware
  943,438,752.62  Insurance
1,452,852,802.28  Payroll
1,560,901,264.14  Recruiting
  492,191,502.64  Rent
  163,891,000.65  Software
  188,161,784.39  Training
  200,935,581.93  Utilities


In [15]:
from dbConfig import create_conn

def get_media_gaming_profit():
    conn = create_conn()
    cursor = conn.cursor()

    query = """
        SELECT 
            r.Month,
            r.TotalRevenue,
            IFNULL(e.TotalExpenses, 0) AS TotalExpenses,
            (r.TotalRevenue - IFNULL(e.TotalExpenses, 0)) AS Profit
        FROM
        (
            SELECT 
                DATE_FORMAT(s.SalesDate, '%Y-%m') AS Month,
                SUM(s.Weekly_Sales) AS TotalRevenue
            FROM sales s
            JOIN departments d ON s.Dept = d.Dept
            WHERE d.DeptDescription LIKE '%Media%' 
               OR d.DeptDescription LIKE '%Gaming%'
            GROUP BY Month
        ) r
        LEFT JOIN
        (
            SELECT 
                DATE_FORMAT(ea.ExpenseDate, '%Y-%m') AS Month,
                SUM(ea.AllocatedAmount) AS TotalExpenses
            FROM expense_allocation ea
            JOIN departments d ON ea.Dept = d.Dept
            WHERE d.DeptDescription LIKE '%Media%'
               OR d.DeptDescription LIKE '%Gaming%'
            GROUP BY Month
        ) e
            ON r.Month = e.Month
        ORDER BY r.Month;
    """

    cursor.execute(query)
    results = cursor.fetchall()

    print("-" * 70)
    print("{:<10} {:>15} {:>15} {:>15}".format("Month", "Revenue", "Expenses", "Profit"))
    print("-" * 70)

    for row in results:
        print("{:<10} {:>15,.2f} {:>15,.2f} {:>15,.2f}".format(
            row[0], row[1], row[2], row[3]
        ))
    print("-" * 70)

    cursor.close()
    conn.close()

if __name__ == "__main__":
    get_media_gaming_profit()

----------------------------------------------------------------------
Month              Revenue        Expenses          Profit
----------------------------------------------------------------------
2019-02       3,981,645.25   16,295,911.34  -12,314,266.08
2019-03       3,312,521.48   20,811,393.35  -17,498,871.87
2019-04       3,443,900.76   20,859,050.62  -17,415,149.86
2019-05       2,422,348.82   20,902,128.31  -18,479,779.50
2019-06       2,597,272.14   20,013,404.23  -17,416,132.09
2019-07       3,052,202.36   16,244,118.01  -13,191,915.64
2019-08       2,280,959.77   15,110,532.30  -12,829,572.52
2019-09       2,276,537.94   15,522,406.60  -13,245,868.66
2019-10       2,974,144.17   15,583,526.02  -12,609,381.85
2019-11       5,440,283.51   15,622,662.84  -10,182,379.34
2019-12       9,601,961.32   15,671,640.69   -6,069,679.37
2020-01       2,360,557.59   14,258,563.27  -11,898,005.67
2020-02       3,113,559.88   10,421,646.24   -7,308,086.36
2020-03       2,856,570.98    9,